# Ridesharing

## Description
The ridesharing problem explores a model routing decisions on a closed network.  Here, there are a fixed number of circulating resources (cars) within a network (represented as a graph) to respond to incoming demands (specified by (source, destination) pair). Specifically, an agent is required to fulfill customer requests, where customers ask for a ride from one location to another. With limited flexibility, the agent can choose a car at a particular location to respond to the request for each demand. The customer may reject the offer probabilitistically based on how far the designated car is from them. If no cars are available or the customer rejects the offer, the request is lost and the agent is penalized accordingly. If the agent successfully designates a car to respond and the customer accepts, the car will be relocated to the destination, so-called the "sink" of the request after a delay, considering the time in transit. The goal of the agent is to minimize the proportion of lost requests and optimizes the cars' movement around the network. The ridesharing environment is offered in two different version, one assuming the travel, the process of serving a demand unit, to be instananeous and another considering travel time of the car from the source to the sink. The latter will be called the travel_time version.

## Model Assumptions 

* New calls do not arrive while a demand is being serviced.

* The supply units relocate instantaneously in during pick up. 


## Dynamics

### State Space

The network is specified via nodes $V$ and edges between the nodes $E$, and each car can be located at any node $v \in V$ (and multiple cars can be at the same node). Moreover, each request is classified via a (source, sink) pair $ \in V \times V$.  Hence, the state space of this environment is specified as $S = [K]^{|V|} \cup V \cup V$, where $K$ is the number of cars.  The first component corresponds to the number of cars at each location in the graph, and the last two components to the current (source, sink) request.

For the travel_time version, there will be an additional transition space where the arrival location and the remaining time of the transition is saved for every car in transition. Therefore, the state space of is specified as $S =[K]^{|V|} \cup (V, [R])^{|K|}  \cup V \cup V,$ where $R$ is the maximum travel time. The added second component corresponds to the transit state, where the arrival location and remaining time for the cars in transition is stored. The third component corresponds to the number of cars in transition.


### Action space

The agent chooses a node where the car responding to the request will depart from. The location for any car can be any node $v \in V$, so the action space $A$ will be $A = V$.


### Reward

When the car chosen by the agent is accepted, the reward is $(f - c) \times d(s_1, s_2) - c \times d(a, s_1) $.   Here $(s_1,s_2)$ is the incoming request (source, sink), $a$ is the node chosen by the agent, and $d$ is the shortest distance between two nodes. $f$ and $c$ is user-defined fare (customer pays per one unit of distance) and cost (the agent pays for per one unit of distance) for traveling unit length of distance, respectively. If the customer rejects the selected car, the reward is $0$. When there were no available cars at the node agent chose, the reward is $-10000 \times \text{MAX DIST} \times c$ where $\text{MAX DIST}$ corresponds to maximum distance in the network. The goal of the agent is to maximize this reward.


### Transitions

Given the incoming reqest $(s_1, s_2) \in V \times V$, and agent's decision $a \in V$, the request is either accepted or rejected with the probability of $ 1/(1+E)$ where $E = e^{-\gamma ( d(s_1, s_2)- d_{threshold})}$. $\gamma$ and $d_{threshold}$ are user-defined parameters that determine how randomly the customer denies the dispatched car and at which value of distance the customer becomes less likely to accept, respectively. If the request is rejected, the state stays the same and the agent is penalized accordingly. If the request is accepted, the car moves from $a$ to $s_2$ in the next state. In other words, the number of cars at $s_2$ is incremented by $1$ and the number of cars at $a$ is decremented by $1$.  The last two components of the state $(s'_1, s'_2)$ are then updated, sampled from the `request_dist` specifying the request distribution.


For the travel_time version, instead of immediately incrementing the number of cars available at the sink $s_2$, there exists a temporary delay, as the car is in transit. Specifically, when a dispatched car is accepted, the car will move from its current location to a transit state where the sink location and the remaining time is saved. The remaining time will be determined set as $d(s_1, s_2) / \text{velocity} $, and decreased for each new request. When the remaining time reaches $0$, the car will be moved from the transit state to the sink location, incrementing the number of cars available at the sink. 

#### Configuration Parameters

* `epLen`: The int number of time steps to run the experiment for.
* `edges`: The array representing edges connecting the nodes and the travel time of each edge.
* `starting_state`: The int array representing the number of available cars at each node.
* `num_cars`: The int number of all cars available in the system.
* `request_dist`: The int tuple representing the source and the sink of the request.
* `reward`: The function ouputting the reward for accepted service.
* `reward_denied`: The function outputting the reward for denied service.
* `reward_fail`: The function outputing the reward for failing to dispatch a car (choosing a location where no car is available).
* `fare` : The amount of reward the agent receives for serving the demand of distant 1
* `cost` : The amount of reward the agent has to pay for moving a car for a distant of 1
* `velocity` : The float number of the parameter for $velocity$.
* `gamma`: The float number of the parameter for $\gamma$. 
* `d_threshold`: The float number of the parameter of $d_{threshold}$.

## Environment


`reset`

Returns the environment to its original state.

`step(action)`

Takes an action from the agent and returns the state of the system after the next arrival.
* `action`: the index of the node where the car agent chose is located

Returns:

* `state`: The array containing the number of available cars at each location

* `reward`: The reward associated with the most recent action and event

* `done`: The boolean of whether the episode terminated

* `info`: a dictionary containing two nodes, each corresponding to the source and the sink of the next request.

`render`

Currently unimplemented

`close`

Currently unimplemented


## Heuristic Agents

### Random Car Agent
The random car agent is a state-dependent assignment policy that will sample a random car and designate it. It differs from the Random Agent, which samples a location from the action space (all the locations the cars can be located at) and can choose a location where no cars are available. More specifically, it will take a weighted random choice from the action space $A = V$ where the weight of node $x \in V$ is the number of cars at $x$.

### Closest Car Agent
The closest car agent will a state-dependent assignment policy that will designate a car that is closest to the source of the request to respond. In other words, it will choose node $x$ such that $x = \text{argmax}_{v \in V} d(v, s_1)$. As the probability of the dispatched car being denied is only dependent on the distance between the source and the chosen car, the agent will always maximize the probability of being accepted.

### MaxWeight Agent
The Maxweight agent is a state-dependent assignment policy which takes a parameter $\alpha$, which is a vector of scaling factors for each location. $i.e., \alpha \in R^{|V|}$ such that $ \alpha _{i} \geq 0$ for all $i \in |V|$. The agent will choose a location $x \in V$ such that $x = \text{argmax}_{v \in V} s[v] \times \Pr[v] \times \alpha_v$, where $s[v]$ represents the number cars at $v$ and $\Pr[v]$ represents the probability that a car dispatched from $v$ will be accepted.
